In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV, LassoLarsIC
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [8]:
# Part (a): Function to calculate required sigma^2 for given R^2

def calculate_sigma2(X_cov, beta_star, r_square):
    """
    Calculate sigma^2 to achieve desired R^2
    Args:
        X_cov: design matrix
        beta_star: true coefficients
        r: desired R^2 value
    Returns:
        sigma2: required noise variance
    """

    return (1 - r_square) * beta_star.T @ X_cov @ beta_star / r_square

# Generate data function

def generate_data(n, p, rho, beta_type='sparse', seed=42):
    """
    Generate synthetic data
    Args:
        n: sample size
        p: dimension
        rho: correlation parameter
        beta_type: 'sparse' or 'dense'
    Returns:
        X: design matrix
        y: response vector
        beta_star: true coefficients
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate correlation matrix Σ(ρ)
    Sigma = np.zeros((p, p))
    for i in range(p):
        for j in range(p):
            Sigma[i, j] = rho ** abs(i-j)

    # Generate X
    X = np.random.multivariate_normal(np.zeros(p), Sigma, size=n)

    # Generate beta_star based on type
    if beta_type == 'sparse':
        beta_star = np.array(
            [2 / np.sqrt(n) if j < np.sqrt(p) else 0 for j in range(1, p + 1)])
    elif beta_type == 'dense':  # dense
        beta_star = np.array([5 / j * np.sqrt(n) for j in range(1, p + 1)])
    else:
        raise ValueError(f"Invalid beta_type: {beta_type}")

    return X, Sigma, beta_star

# Model fitting function with different tuning methods


def fit_model(X, y, model_type, tuning_method):
    """
    Fit model with specified tuning method
    Args:
        X: design matrix
        y: response vector
        model_type: 'Lasso', 'Ridge', 'Adaptive Lasso', or 'Adaptive Ridge'
        tuning_method: 'AIC', 'BIC', or 'LOO-CV'
    Returns:
        fitted model
    """
    if model_type == 'Lasso':
        if tuning_method == 'LOO-CV':
            model = LassoCV(cv=len(X))
        elif tuning_method in ['AIC', 'BIC']:
            model = LassoLarsIC(criterion=tuning_method.lower())
        model.fit(X, y)

    elif model_type == 'Ridge':
        alphas = np.logspace(-6, 6, 100)
        if tuning_method == 'LOO-CV':
            model = RidgeCV(alphas=alphas, cv=len(X))
        elif tuning_method in ['AIC', 'BIC']:
            # For Ridge with AIC/BIC, we'll use RidgeCV with cross-validation
            model = RidgeCV(alphas=alphas, cv=5)
        model.fit(X, y)

    elif model_type == 'Adaptive Lasso':
        # First fit Ridge to get initial weights
        lasso = Lasso(alpha=1.0)
        lasso.fit(X, y)
        weights = 1 / np.abs(lasso.coef_ + 1e-6)
        weights = weights / np.mean(weights)
        X_weighted = X * weights

        if tuning_method == 'LOO-CV':
            model = LassoCV(cv=len(X))
        elif tuning_method in ['AIC', 'BIC']:
            model = LassoLarsIC(criterion=tuning_method.lower())

        model.fit(X_weighted, y)
        # Store weights for prediction
        model.weights = weights

    elif model_type == 'Adaptive Ridge':
        # First fit OLS to get initial weights
        ridge = Ridge(alpha=1.0)
        ridge.fit(X, y)
        weights = 1 / np.abs(ridge.coef_ + 1e-6)
        weights = weights / np.mean(weights)
        X_weighted = X * weights

        alphas = np.logspace(-6, 6, 100)
        if tuning_method == 'LOO-CV':
            model = RidgeCV(alphas=alphas, cv=len(X))
        elif tuning_method in ['AIC', 'BIC']:
            model = RidgeCV(alphas=alphas, cv=5)

        model.fit(X_weighted, y)
        # Store weights for prediction
        model.weights = weights

    else:
        raise ValueError(f"Invalid model_type: {model_type}")

    # Add predict method for adaptive models
    if model_type in ['Adaptive Lasso', 'Adaptive Ridge']:
        original_predict = model.predict
        def new_predict(self, X_new):
            return original_predict(X_new * model.weights)
        model.predict = new_predict.__get__(model)

    return model

def run_simulation(n, p, rho, beta_type, n_datasets=10):
    """
    Run full simulation for given parameters
    """
    results = []

    for dataset in tqdm(range(1, n_datasets + 1)):
        # Generate data
        X, X_cov, beta_star = generate_data(n, p, rho, beta_type)

        # Calculate required sigma^2 for R^2 = 0.8
        sigma2 = calculate_sigma2(X_cov, beta_star, 0.8)

        # Generate response with proper noise
        epsilon = np.random.normal(0, np.sqrt(sigma2), n)
        y = X @ beta_star + epsilon

        # Fit all models
        models = [
            (i, j)
            for i in ['Lasso','Adaptive Lasso', 'Ridge', 'Adaptive Ridge']
            for j in ['AIC', 'BIC', 'LOO-CV']
        ]

        for model_type, tuning in models:
            try:
                model = fit_model(X, y, model_type, tuning)
                mse = mean_squared_error(y, model.predict(X))
                results.append({
                    'model': model_type,
                    'tuning': tuning,
                    'mse': mse,
                    'n': n,
                    'p': p,
                    'rho': rho,
                    'beta_type': beta_type
                })
            except Exception as e:
                print(
                    f"Error in simulation {dataset} with {model_type}, {tuning}, {n}, {p}, {rho}, {beta_type}: {str(e)}")

    return pd.DataFrame(results)

In [9]:
# Run simulations for all parameter combinations
parameter_settings = [
    {'n': 100, 'p': p, 'rho': rho, 'beta_type': beta_type}
    for p in [10, 25, 50]
    for rho in [0, 0.25, 0.5]
    for beta_type in ['sparse', 'dense']
]

# Store results
all_results = []
for params in parameter_settings:
    results = run_simulation(**params)
    all_results.append(results)

# Combine and analyze results
final_results = pd.concat(all_results, axis=0).reset_index(drop=True)
print(final_results)

average_results = (final_results
    .groupby(['model', 'tuning', 'p', 'rho', 'beta_type'])['mse']
    .mean()
    .reset_index()
)

# Create a more readable table format
table = average_results.pivot_table(
    index=['p', 'rho', 'beta_type'],
    columns=['model', 'tuning'],
    values='mse'
)

# Print the formatted table
print("\nAverage MSE across 1000 simulations:")
print(table)

# Optionally, save to CSV
# table.to_csv('simulation_results.csv')

100%|██████████| 10/10 [01:26<00:00,  8.62s/it]

               model  tuning         mse    n   p  rho beta_type
0              Lasso     AIC    0.026246  100  10  0.0    sparse
1              Lasso     BIC    0.027106  100  10  0.0    sparse
2              Lasso  LOO-CV    0.026431  100  10  0.0    sparse
3     Adaptive Lasso     AIC    0.026246  100  10  0.0    sparse
4     Adaptive Lasso     BIC    0.027106  100  10  0.0    sparse
...              ...     ...         ...  ...  ..  ...       ...
2155           Ridge     BIC  883.274102  100  50  0.5     dense
2156           Ridge  LOO-CV  744.950331  100  50  0.5     dense
2157  Adaptive Ridge     AIC  745.011022  100  50  0.5     dense
2158  Adaptive Ridge     BIC  745.011022  100  50  0.5     dense
2159  Adaptive Ridge  LOO-CV  744.950464  100  50  0.5     dense

[2160 rows x 7 columns]

Average MSE across 1000 simulations:
model             Adaptive Lasso                            Adaptive Ridge  \
tuning                       AIC          BIC        LOO-CV            AIC   
p